# Исследование надёжности заёмщиков <a id='begin'></a>

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Содержание отчета:** 
1. [Знакомство с датасетом](#intro)
2. [Предобработка данных](#fix)
   * [Обработка пропусков](#empty)
   * [Замена типа данных](#type)
   * [Обработка дубликатов](#dup)
   * [Лемматизация](#lem)
   * [Категоризация данных](#cat)
3. [Ответы на вопросы](#qs)
   * [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#q1)
   * [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#q2)
   * [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#q3)
   * [Как разные цели кредита влияют на его возврат в срок?](#q4)
4. [Общий вывод](#total)


## Знакомтсво с датасетом
<a id='intro'></a>
[в начало](#begin)

Расмотрим датасет: Статистика платежоспособности клиентов - `data.csv`

In [1]:
# Ячейка для включения дополнительных модулей (будет дополняться по мере необходимости)
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

In [2]:
data = pd.read_csv('/datasets/data.csv')

Для того, чтобы описать данные, используемые в датасете, выведем первые пять строк сформированного Датафрейма.

In [3]:
# Вывод первых пять строк датафрейма data
# Выполним с помощью атрибута head()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Описание данных:**
* **children** - количество детей в семье
* **days_employed** - трудовой стаж в днях
* **dob_years** - возраст клиента в годах
* **education** - уровень образования клиента
* **education_id** - идентификатор уровня образования
* **family_status** - семейное положение
* **family_status_id** - идентификатор семейного положения
* **gender** - пол клиента (**F** - женский / **M** - мужской)
* **income_type** - тип занятости
* **debt** - имел ли задолженность по возврату кредитов
* **total_income** - ежемесячный доход
* **purpose** - цель кредита

**Предварительный вывод**: 
- наименования столбцов стилистически составлены корректно
- для некоторых столбцов, например, **family_status_id**, **education_id** отсутствует информация, которая раскрывает смысл значений в этих столбцах.
- некорректное значение в столбце **total_income**, значения в этом столбце имеет шесть знаков после запятой 
- некорректное значение в столбце **days_employed**, значения есть как положительные, так и отрицательные; непонятен механизм расчета трудового стажа в днях
- некорреткное представление значений в группе **education**, так как используется прописные/строчные буквы в уровнях образования
- в столбце **family_status** имеются одинаковые по смыслу значения, но представлены они как разные элементы категории. Например: гражданский брак - женат/замужем

**Предварительное решение проблемы:**
- обратиться к тому, кто сформировал этот датасет и узнать смысл значений в столбцах, где не хватает расшифровки значений;
- разобраться самому: найти закономерность между значениями некоторых столбцов и убедиться в их взаимосвязи
- разобраться с размерностью в столбце **total_income** в каких единицах измерения представлен объект, привести данное значение к целочисленному типу
- разобраться с размерностью в столбце **days_employed** как считается подсчет дней, определить закономерность между положительным и отрицательным значениями в столбце, определить: применяется ли этот столбец в исследовании?

Рассмотрим общую информацию о полученной таблице.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Предварительный вывод:** в столбцах **days_employed**, **total_income** присутствуют явные пропуски в данных.

**Предварительное решение:** необходимо провести обработку пропусков

**Вывод:** после знакомства с общей информацией датафрейма можно заметить следующее:
* имеются явные пропуски в столбцах (**days_employed**, **total_income**)
* отсутствует дополнительная информация по некоторым столбцам (**family_status_id**, **education_id**) 
* имеются неявные дубликаты в столбце **education** 
* данные в столбцах **days_employed** и **total_income** заполнены некорректно
* данные в столбце **family_status** представляют одну категорию, но названы по-разному

Данный набор данных не готов для проведения анализа и решения поставленных задач.

**Решение:** необходимо провести предварительную обработку данных (убрать дубликаты, привести к соответсвтующему типу, заполнить или устранить пропуски в данных, провести объединение )

## Предобработка данных <a id='fix'></a>
[в начало](#begin)

### Обработка пропусков <a id='empty'></a>
[в начало](#begin)

Прежде чем начать обрабатывать пропуски в данных, мы должны определить следующее:
* к какой категории относится признак, в котором присутствует пропуск? 
* какой метод обработки пропуска использовать для признаков?

Прежде чем начать обработку данных, рассмотрим уникальные значения в **категориальных** переменных. К ним относятся следующие признаки:
* **education**
* **education_id**
* **family_status**
* **family_status_id**
* **gender**
* **income_type**
* **debt**
* **purpose**
* **

Для столбца **gender:**

In [5]:
display(data['gender'].unique())

array(['F', 'M', 'XNA'], dtype=object)

Как оказалось, в столбце **gender** есть три значения. Посчитаем количество упоминаний каждого из них.

In [6]:
display(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

А теперь выведем строку, где значение в столбце **gender** является пустым:

In [7]:
display(data[data['gender'] == 'XNA'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


**Предвариательный вывод:** Данный пропуск можно считать полностью случайным, так как возникновение этого пропуска не зависит от каких-либо других значений. 

Возможные причины, по которой появился данный пропуск:
* **Человеческий фактор:** сотрудник банка забыл дописать значение "пол" в данных клиента банка; клиент банка забыл указать о себе эту информацию при заполнении заявки.

**Возможное решение:** удалить строку с пропуском с изменением порядка индексов. Так как один результат из 21525 наблюдений представляет собой 1 /21525 = 0.00004 * 100% = 0.004%. Исключение столбца из этого наблюдения существенно не исказит результаты исследования 

In [8]:
# Присваиваем строке со значением XNA значение None
# Удаляем строку с этим значением из исходной таблицы 
# изменяем порядок индекса
data.loc[data['gender'] == 'XNA', 'gender'] = None
data = data.dropna(subset = ['gender']).reset_index(drop = True)

Убедимся в том, что удалили строку с пропуском в столбце **gender**:

In [9]:
# Проверка удаления строки 
display(data['gender'].value_counts())

F    14236
M     7288
Name: gender, dtype: int64

**Предварительный вывод:** строка с пропуском в признаке **gender** удалена, порядок индексов был изменен. Работа с пропусками в категориальных переменных заврешена.

Теперь перейдем к **количественным** переменным. К ним относятся следующие признаки:
* **chidren**
* **days_employed**
* **dob_years**
* **total_income**

Рассмотрим еще раз общую информацию о данных в датафремйе:

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       19350 non-null float64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        19350 non-null float64
purpose             21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Явные пропуски присутствуют в следующих признаках: **days_employed** и **total_income**.
Однако предполагаем, что в данных присутствуют неявные пропуски.

Рассмотрим признак **dob_years**. Для этого выведем список сортированных по возрастанию уникальных значений этого признака.

In [11]:
# Сортируем значение столбца dob_years по возрастанию и выбираем уникальные отсортированные значения этого столбца
years_list = data.sort_values(by = 'dob_years', ascending = True)['dob_years'].unique()
display(years_list)

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

Выведем количество строк в таблице, где значение **age** равен нулю:


In [12]:
display(data[data['dob_years'] == 0]['children'].count())

101

Выведем первые пять строк таблицы, где значение **age** равен нулю для поиска закономерности:

In [13]:
display(data[data['dob_years'] == 0].head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль


**Предварительный вывод:** в столбце **age** есть значения, которые, предположительно, являются неявными пропусками. Так как человек не может брать кредит с возрастом в 0 лет. Закономерность в связи между неявным пропуском в возрасте клиента и остальными признакми отсутствует.

Возможные причины, по которой появился данный пропуск:

* **Человеческий фактор:** клиент банка забыл указать или намеренно скрыл о себе эту информацию при заполнении заявки; специалист, случайно пропустил данный пункт про оформлении заявки.
* **Техничский фактор:** система сбора данных при обработке значений могла дать сбой и заполнила некоторые строки столбца **age** нулями; произошла фильтрация пользователей, возраст которых равен 18. Однако, если сопоставить возраст с должностью, то можно заметить, что есть наблюдения, где возраст равен нулю, а должность - пенсионер. Поэтому предположим, что с точки зрения технического фактора, причина пропуска была связана с ошибкой заполнения нужного столбца

**Предварительное решение:** 
* **консультация** с разработчиком для уточнения причины заполнения пропуска нулями
* **фильтрация** исходной таблицы по возрасту (т.е. исключить из исходной таблицы строки с нулевым значением), так как количество полей с неявным пропуском равно 101, а для выборки в 21454 строк данная фильтрация не исказит результаты исследования (101 / 21454 = 0.004 * 100% = 0.4%) данное количество значений существенно мало.
* **заполнение пропуска средним арифметическим значением возраста клиентов**, значения, которые нам понадобятся для исследования не будут утеряны и можно составить более точный анализ, так как в вопросах исследования не учитывается возраст при проведении анализа.

Остановимся на третьем варианте решения проблемы.


Посчитаем среднее арифметическое значение по возрасту без учета количества клиентов, возвраст которых равен нулю.

In [14]:
#avg_age_filtered - переменная содержит в себе среднее арифметическое значение возраста клиентов без учета неявного пропуска
avg_age_filtered = int(data[data['dob_years'] != 0 ]['dob_years'].mean())

Теперь заменим "нулевой" возраст клиентов на посчитанное среднее арифметическое значение возраста.

In [15]:
data.loc[data['dob_years'] == 0, 'dob_years'] = avg_age_filtered

Удостоверимся в изменении нашей таблицы. Выведем список уникальных значений столбца **age**.

In [16]:
display(data.sort_values(by = 'dob_years', ascending = True)['dob_years'].unique())

array([19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75])

**Предварительный вывод:** неявные пропуски в столбце **gender** были заменены на средний арифметический возраст клиентов. 101 наблюдение, которые могли повлиять на окончательный результат исследования сохранены.

Проверим столбец **debt**.

In [17]:
display(data['debt'].value_counts())

0    19783
1     1741
Name: debt, dtype: int64

Здесь все хорошо, идем дальше.

Проверим столбец **children**:

In [18]:
display(data['children'].value_counts())

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Здесь мы видим два аномальных значения: -1 и 20. Выведем сроки для каждого аномального случая для выявления закономерности, а также сгруппируем их по разынм признакам и посчитаем количество строк.

Для `-1`:

In [19]:
data_1 = data[data['children'] == -1]
data_1.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,NaN,57,Среднее,1,женат / замужем,0,F,пенсионер,0,NaN,на покупку своего автомобиля


In [20]:
display(data_1.groupby('income_type')['children'].count().sort_values(ascending = False))

income_type
сотрудник      26
компаньон       9
пенсионер       8
госслужащий     4
Name: children, dtype: int64

In [21]:
display(data_1.groupby('gender')['children'].count())

gender
F    35
M    12
Name: children, dtype: int64

In [22]:
display(data_1.groupby('education')['children'].count().sort_values(ascending = False))

education
среднее                31
высшее                  8
СРЕДНЕЕ                 4
Среднее                 2
неоконченное высшее     1
ВЫСШЕЕ                  1
Name: children, dtype: int64

In [23]:
display(data_1.groupby('family_status')['children'].count().sort_values(ascending = False))

family_status
женат / замужем          29
гражданский брак          5
Не женат / не замужем     5
вдовец / вдова            4
в разводе                 4
Name: children, dtype: int64

Теперь для `20`:

In [24]:
data_20 = data[data['children'] == 20]
data_20.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,43,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля


In [25]:
display(data_20.groupby('income_type')['children'].count().sort_values(ascending = False))

income_type
сотрудник      43
компаньон      22
пенсионер       9
госслужащий     2
Name: children, dtype: int64

In [26]:
display(data_20.groupby('gender')['children'].count())

gender
F    47
M    29
Name: children, dtype: int64

In [27]:
display(data_1.groupby('education')['children'].count().sort_values(ascending = False))

education
среднее                31
высшее                  8
СРЕДНЕЕ                 4
Среднее                 2
неоконченное высшее     1
ВЫСШЕЕ                  1
Name: children, dtype: int64

In [28]:
display(data_1.groupby('family_status')['children'].count().sort_values(ascending = False))

family_status
женат / замужем          29
гражданский брак          5
Не женат / не замужем     5
вдовец / вдова            4
в разводе                 4
Name: children, dtype: int64

**Предварительный вывод**: аномальные значения в этом столбце не имеют закономерности, однако имеют схожую частоту заполнения. В большинстве случаев, этим значениями заполняются наблюдения, связанные с сотрудниками, людьми женского пола, людьми со средним образованием, замужем/женат. Подобный пропуск может быть связан со следующим фактором:
* **Технический фактор**: пропуск заполняется -1, если пункт был проигнорирован клиентов или написан в форме (нет детей), значение 20 - когда клиент при оформлении заявки написал: "есть дети", и не совсем ясно, какое количество детей имелось в ввиду.

**Предварительное решение**
* для случая -1: заменить значение на 0, так можно убедить в том, что мы корректно представляем наши данные в таблице.
* для случая 20: посчитать с учетом замены -1 на медианное значение количество детей, чтобы отбросить аномальное количество детей.
* проконсультироваться с разработчиком о корректности заполнения данных.
* проигнорировать строчки, где есть есть значения -1 и 20, так как эти значения вместе представляют собой существенно малую выборку (123 строчки) и их исключение несущественно исказит результаты исследования.

Выберем последний вариант.

In [29]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Проверим полученные изменения в таблице:

In [30]:
print(data['children'].value_counts())

0    14148
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


Аномальные значения проигнорированы, идем дальше. 

Вернемся к явным пропускам в столбцах **days_employed** и **total_income**. 

Посчитаем количество пропусков в столбцах и выведем таблицу, в котором данные столбцы заполнены пропусками для определения причины возникновения пропуска.

In [31]:
print('Количество пропусков в столбце days_employed:', data['days_employed'].isna().sum())

Количество пропусков в столбце days_employed: 2162


In [32]:
print('Количество пропусков в столбце total_income:', data['total_income'].isna().sum())

Количество пропусков в столбце total_income: 2162


In [33]:
data[ (data['days_employed'].isna()) &  (data['total_income'].isna())].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [34]:
# Подсчет количества строк, в которых столбцы total_income и days_employed одновременно пусты
print('Количество "пустых" строк:', data[ (data['days_employed'].isna()) &  (data['total_income'].isna())]['dob_years'].count())

Количество "пустых" строк: 2162


In [35]:
data_nan_values = data[ (data['days_employed'].isna()) &  (data['total_income'].isna())]
display(data_nan_values.groupby('education')['dob_years'].count().sort_values(ascending = False))

education
среднее                1400
высшее                  493
СРЕДНЕЕ                  67
Среднее                  64
неоконченное высшее      55
Высшее                   25
ВЫСШЕЕ                   23
начальное                19
Неоконченное высшее       7
НЕОКОНЧЕННОЕ ВЫСШЕЕ       7
Начальное                 1
НАЧАЛЬНОЕ                 1
Name: dob_years, dtype: int64

**Предварительный вывод:** обнаружена закономерность: пропуски в столбцах **days_employed** и **total_income** присутствуют одновременно. Большинство пропусков связана со средним и высшим образованиями. Пропуски можно считать случайными.
* **Человеческий фактор:** люди со средним и высшим образованиями склонны скрывать информацию о своем стаже и ежемесячном доходе; возможна ситуация, когда сотруник банка некорректно заполнял данные клиентов (и такое бывает, наверное).

**Предвариательное решение:**
* **Исключить** строки, где присутствуют пропуски в значениях. Это может повлиять на окончательный результаты исследования, так как пропуски представляют собой 2164 / 21524 = 0.1 * 100 % = 10% выборки. Удалять строки с пропусками нельзя.
* **Исключить** столбец **days_employed**. Так как этот столбец для проведения исследования не нужен, а его удаление никак не повлияет на результаты исследования. А также **заменить** пропуски в столбце **total_income** на медианное значение ежемесячного дохода выборки. 
* **Уточнить** у разработчика: корректно ли заполняются значения в этих столбцах.

Выберем второй вариант решения проблемы.

Для начала заполним значения столбца **total_income** медианным значением.

In [36]:
total_income_median = data['total_income'].median()
try:
    data['total_income'] = data['total_income'].fillna(total_income_median)
except:
    print("Что-то пошло не так!")

Проверим, заполинились ли пропуски в этом столбце. Для этого посмотрим общую информацю о таблице.

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21401 entries, 0 to 21523
Data columns (total 12 columns):
children            21401 non-null int64
days_employed       19239 non-null float64
dob_years           21401 non-null int64
education           21401 non-null object
education_id        21401 non-null int64
family_status       21401 non-null object
family_status_id    21401 non-null int64
gender              21401 non-null object
income_type         21401 non-null object
debt                21401 non-null int64
total_income        21401 non-null float64
purpose             21401 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Явные пропуски в столбце **total_income** устранены.
Теперь удалим столбец **days_employed**.

In [38]:
data = data.dropna(axis = 'columns')

Проверим, удалился ли столбец с **days_employed**:

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21401 entries, 0 to 21523
Data columns (total 11 columns):
children            21401 non-null int64
dob_years           21401 non-null int64
education           21401 non-null object
education_id        21401 non-null int64
family_status       21401 non-null object
family_status_id    21401 non-null int64
gender              21401 non-null object
income_type         21401 non-null object
debt                21401 non-null int64
total_income        21401 non-null float64
purpose             21401 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 2.0+ MB


Столбец удален, пропуски устранены.

**Вывод:** в исходных данных присутствовали пропуски (явные и неявные), как в категориальных, так и в количественных переменных. Причины возникновения этих пропусков связаны как с человеческим фактором (преднамеренное сокрытие данных, ошибка при заполнении заявки клиента), так и техническим фактором (некорректная обработка значений). Были рассмотрены возможные причины их возникновения, а также возможные варианты решения сформированных проблем. Таблица готова для дальнейшей предварительной обработки данных.

### Замена типа данных <a id='type'></a>
[в начало](#begin)

Прежде чем заменить данные, рассмотрим какой тип данных используется к каждому признаку.

In [40]:
data.dtypes

children              int64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

Рассмотрим первые пять строк таблицы, чтобы понять, какие значения имеет столбец **total_income**.

In [41]:
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Предварительный вывод:** нас интересует столбец **total_income** - ежемесячный доход. Значения этого столбца имеют тип "с плавающей точкой", т.е. `float64`. Сами значения имеют шесть и более знаков после деления, что делает представление о ежемесячном доходе каждого клиента некорректным. Определить как, почему и откуда заполняются в такой форме данные невозможно, так как не хватает информации. Возможная причина некорректного заполнения столбца:
* **Технический фактор:** Разработчик некорректно заполняет столбец. Не существует миллионой доли денежного эквивалента (только до второго знака после запятой). Более того, пункт денежнего дохода в контексте решения задачи представляет собой доход без учета копеек.

**Возможное решение:** 
* заменить тип столбца **total_income** на `int`
* уточнить у разработка корректность заполнения значений в этот признак

Проведем замену типа значений столбца **total_income** на `int`. Для этого воспользуемся методом `astype()` для преобразования типа в столбце на целочисленный:

In [42]:
try:
    data['total_income'] = data['total_income'].astype('int')
except:
    print('Error!')
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Выведем общую информацию о типах данных в таблице:

In [43]:
data.dtypes

children             int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

Тип данных в столбце **total_income** заменен на целочисленный тип.

**Вывод:** провели замену типа данных с `float` на `int` в столбце **total_income**. Некорректный к применению тип данных этого признака исказал представления ежемеячного дохода каждого клиента.

### Обработка дубликатов <a id='dup'></a>
[в начало](#begin)

Прежде чем начать обработку дубликатов, посчитаем общее количество дубликатов в таблице:

In [44]:
print('Количество дубликатов:', data.duplicated().sum())

Количество дубликатов: 54


Выведем таблицу, строки которые являются явными дубликатами:

In [45]:
data_duplicated = data[data.duplicated()]
data_duplicated.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
4182,1,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145017,свадьба
4851,0,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба
5557,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
7808,0,57,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы


Выведем наименования уникальных значений столбца **total_income** и количество их упоминаний.

In [46]:
display(data['total_income'].value_counts())

145017    2163
177986       3
145603       3
133122       3
244123       3
          ... 
60407        1
156662       1
130037       1
379891       1
264193       1
Name: total_income, Length: 18501, dtype: int64

Значение `145017` - это резльутат заполнения пропуска медианным значением ежемесячного дохода клиентов.

Прежде всего обратим внимание на столбец **education**.
- в первом случае: столбец содержит в себе значения строкового типа разного регистра (верхний/нижний)

Рассмотрим уникальные значения этих столбцов.

Для столбца **education**:

In [47]:
display(data['education'].unique())

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Для столбца **family_status**:

In [48]:
display(data['family_status'].unique())

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

**Предварительный вывод:** в исходных данных содержится 54 дубликат. Кроме того, в значениях столбца **education** содержатся значения строкого типа разного регистра. Возможные причины возникновения дубликатов:
* **Технический фактор**: Заполнение пропуска медианным значением в таблице, предположительно, дал такой результат. Однако наблюдения, указанные в таблице с дубликатами, явным дубликатом считать нельзя, так как есть показатели, по которым отличаются наблюдения: возраст, образование, пол, цели кредита клиента. 

**Предварительное решение:**
* **education**: приведение значений столбцов к нижнему регистру. Таким образом, мы избавимся от дубликатов, написанные разными регистрами, затем провести удаление дубликатов в исходной таблице, так как дубликатов в таблице всего 54, а их удаление несущественно повлияет на результаты исследования.

Приведем значения столбцов **education** к нижнему регистру:

In [49]:
 data['education'] = data['education'].str.lower()

Теперь посмотрим на количество дубликатов после приведения к нижнему регистру значений:

In [50]:
print(data.duplicated().sum())

71


Количество дубликатов в исходной таблице увеличилось стало равны 71. Это связано с тем, что некоторые, значения, имеющие другой регистр, объединились с другими значениями, которые могли быть дубликатом. Удаление 71 строки существенно не повлияет на результаты исследования.

Выведем список уникальных значений столбца, чтобы убедиться, что все значения приведены к нижнему регистру: 

In [51]:
display(data['education'].unique())

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Значения столбца **education** приведены к общему виду. Теперь удалим дубликаты из исходной таблицы.

In [52]:
data = data.drop_duplicates().reset_index(drop = True)

Убедимся в том, что все дубликаты из исходной таблицы были удалены.

In [53]:
if data.duplicated().sum() == 0:
    print('Все дубликаты удалены, значение:', data.duplicated().sum())
else:
    print('Остались дубликаты! Значение:', data.duplicated().sum())

Все дубликаты удалены, значение: 0


**Вывод:** из-за того, что пропуски в столбце **total_income** были заполнены медианным значением, появилось 54 дубликата в исходной таблице. После приведения к нижнему регистру значений признака **education**, появилось еще 14 дубликата. Все дубликаты были удалены из исходной таблицы, поскольку их удаление существенно не повлияет на результат исследования.

### Лемматизация <a id='lem'></a>
[в начало](#begin)

Расмотрим первые 15 строк датафрейма `data`:

In [54]:
data.head(15)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


В разделе "Лемматизация" нас интересует столбец **purpose**. 

Рассмотрим уникальные значения этого столбца.

In [55]:
display(data['purpose'].unique()) 

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Предварительный вывод:** Как видно из списка уникальных значений, цели кредита у всех клиентов одинаковые, но формулировка причины - разные.
Для того, чтобы не искать клиентов с разной формулировкой цели кредита, рекомендуется провести лемматизацию каждого значения столбца **lemmas**, т.е. привести каждое значение к лемме. Далее эти леммы можно использовать в разделе [категоризация данных](#category) , чтобы сгруппировать разные формулировки клиентов на категории запроса.

**Предварительное решение:** написать функцию, которая на вход будет получать значения столбца **purpose**, каждое значение которого будет добавляться в виде леммы. Затем с помощью сформированных лемм создать новый столбец с категорией заявок клиента. В данном разделе решения задачи, принял решение показать разбор значений каждого столбца на леммы. Для более подробной реализации категоризации с применением лемматизации будет рассмотрена в разделе "категоризация данных".

In [56]:
purpose_list = [] #создаем список, куда будем добавлять наши леммы построчно
for elem in data['purpose']: #делаем пробег по элементам 
    purpose_lemma = m.lemmatize(elem) #разбираем строку на леммы
    purpose_list.append(purpose_lemma) #добавляем в список

Выведем полученный результат лемматизации.

In [57]:
display(purpose_list)

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['покупка', ' ', 'жилье', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['недвижимость', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'п

Выглядит результат объемным и излишне наглядным, но так постарался показать сам процесс лемматизации - получиили список списков, каждый список - это строка, разобранная на леммы. В качестве более простого решения задачи лемматизация и категоризация данных предлагается следующее: 
* написать функцию, которая будет принимать столбец из датафрейма (в нашем случае - **purpose**)
* далее проводим лемматизацию переданного в функцию столбца
* затем проводим сравнение элементов: если есть лемма в этой строке, то возвращаем новую категорию

Проведем реализацию в следующем разделе исследования.

**Вывод**: в качестве объекта для проведения лемматизации выбрали столбец **purpose**. Клиенты имеют разные цели кредита, а также разные формулировки своих целей. После разбора каждого запроса на леммы, можно перейти к разделу "категоризация данных""

### Категоризация данных <a id='cat'></a>
[в начало](#begin)

Прежде чем провести категоризацию данных, нужно ответить на следующие вопросы:
* Зачем проводить категоризацию? 

Для того, чтобы проще работать с данными, которые мы хотим использовать в исследовании, при этом более наглядно визуализировать результаты.
* Какими значениями заполнить категорию? 

Все зависит от того, что именно мы хотим категоризировать.
* На какие данные опираться при реализации категоризации? 

Рассмотрим решение нашей задачи и по нашим исходным данным.

У нас есть следующие столбцы, которые необходимо использовать в исследовательской части проекта: 
* **children**: используется в решении задачи: есть ли зависимость между **наличием детей** и возвратом кредита в срок? Нам не так важно количество детей, интересует только их наличие (есть/нет). Значения из этого столбца можно категоризировать.
* **family_status**: используется в решении задачи: есть ли зависимость между **семейным положением** и возвратом кредита в срок? Эти данные уже есть в данном столбце, однако у нас есть столбец **family_status_id**, значения которого мы еще не связали с нашим столбцом, это нужно сделать, чтобы проще визуализировать свои результаты.
* **total_income**: используется в решении задачи: есть ли зависимость между **уровнем дохода** и возвратом кредита в срок? В данном случае нужно провести категоризацию диапазонно: то есть, определить каждый элемент категории в виде диапазона значений (от и до)
* **purpose**: используется в решении задачи: как **разные цели** кредита влияют на его возврат в срок? Здесь как раз нам и понадобится наша лемматизация. Дело в том, что благодаря тому, что мы привели каждое значение столбца к лемме, можно составить нужную категоризацию по целям, т.е. объединить схожие по смыслу формулировки в одну категорию.
* **debt**: ключевой критерий, по которому проводят исследования в данном проекте. Для начала нужно определить: какие значения имеет этот столбец? Количество их упоминаний? И составить категоризацию по полученным данным.

Начнем с столбца **children**. Посчитаем уникальные значения и их упоминание:

In [58]:
display(data['children'].value_counts())

0    14090
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Здесь мы видим количество детей: от 1 до 5. Однако в исследовании необходимо учесть только лишь факт наличия детей. Поэтому мы проведем следующую категоризацию по наличию детей:
* **нет детей** - количество детей равно нулю
* **есть дети** - количество детей больше нуля

Для формирования нового столбца для категоризации, составим новый столбец.

In [59]:
def have_children(children):
    if children == 0:
        return 'нет детей'
    else:
        return 'есть дети'
# создаем новый столбец в таблице, и добавляем в него значения из функции have_children  
data['have_children'] = data['children'].apply(have_children)
# проверка выполненной функции:
display(data.head())

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,have_children
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,есть дети
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,есть дети
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,нет детей
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,есть дети
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,нет детей


Категоризацию по наличию детей добавили, идем дальше.

Рассмотрим столбцы **family_status** и **family_status_id**. В данном случае, нам необходимо определить их взаимосвязь между собой. Для этого посомтрим количество упоминаний каждого из них. 

In [60]:
display(data['family_status'].value_counts())

женат / замужем          12261
гражданский брак          4133
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: family_status, dtype: int64

In [61]:
display(data['family_status_id'].value_counts())

0    12261
1     4133
4     2796
3     1189
2      951
Name: family_status_id, dtype: int64

Для проверки выведем количество строк, которые будут объединять оба условия.

In [62]:
family_status_list = data['family_status'].unique()
family_status_id_list = data['family_status_id'].unique()

for elem in range(len(family_status_list)):
    print(f'{family_status_list[elem]} и {family_status_id_list[elem]} вместе:', data[(data['family_status'] == family_status_list[elem]) & (data['family_status_id'] == family_status_id_list[elem])]['children'].count())

женат / замужем и 0 вместе: 12261
гражданский брак и 1 вместе: 4133
вдовец / вдова и 2 вместе: 951
в разводе и 3 вместе: 1189
Не женат / не замужем и 4 вместе: 2796


Таким образом, можно сказать следующее: столбцы **family_status** и **family_status_id** связаны между собой по следующим значениям:
* `0` - женат/замужем
* `1` - гражданский брак
* `2` - вдовец/вдова
* `3` - в разводе
* `4` - не женат/не замужем 

По этим двум значениям, зная их смысл можно теперь проводить исследование, идем дальше

Теперь для категоризации возьмем столбец **total_income**. Для того, чтобы категоризировать по уровню дохода клиентов, воспользуемся [статьей](https://zen.yandex.ru/media/dogecrypto/kak-poschitat-k-kakomu-klassu-vy-otnosites-po-urovniu-dohoda-5b150b5aea0fe700a848e1b1) и категоризируем по указанным в статье доходы клиентов на следующие категории:
* **беднейшие** - меньше 15 т.р.
* **бедные** - от 15 до 18 т.р
* **выше бедности** - от 18 до 36 т.р.
* **нижний средний класс** - от 36 до 100 т.р
* **предсредний класс** - от 100 до 150 т.р.
* **средний класс** - от 150 до 250 т.р.
* **верхний средний класс** - от 250 до 500 т.р.
* **состоятельные** - от 500 т.р до 1 млн.р.
* **богатые** - от 1 млн. р. и выше

In [63]:
# функция на вход принимает значения из столбца total_income и сравнивает их
def take_total_income_category(total_income):
    if total_income <= 15000:
        return 'беднейшие'
    elif total_income in range(15001, 18001):
        return 'бедные'
    elif total_income in range(18001, 36001):
        return 'выше бедности'
    elif total_income in range(36001, 100001):
        return 'нижний средний класс'
    elif total_income in range(100001, 150001):
        return 'предсредний класс'
    elif total_income in range(150001, 250001):
        return 'средний класс'
    elif total_income in range(250001, 500001):
        return 'верхний средний класс'
    elif total_income in range(500001, 1000001):
        return 'состоятельные'
    else:
        return 'богатые'
# заполняем новый столбец с помощью метода apply
data['total_income_category'] = data['total_income'].apply(take_total_income_category) 
# проверка на вывод 
display(data.head())

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,have_children,total_income_category
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,есть дети,верхний средний класс
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,есть дети,предсредний класс
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,нет детей,предсредний класс
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,есть дети,верхний средний класс
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,нет детей,средний класс


Столбец с категориями заполнен, можно идти дальше. 

Теперь рассмотрим столбец **purpose** как было описано в [лемматизации](#lem). Люди формируют одинаковые цели кредита, но по-разному. Для этого предлагается сделать функцию, которая сделает лемматизацию каждого значения, а затем вернет категорию как результат нахождения леммы в лемматизированной цели каждого клиента.

In [64]:
def purpose_category(row):
# лемматизирую каждую строку столбца purpose
    lemma_purpose = m.lemmatize(row['purpose'])
    # проверка слова в лемме и в случае наличия слова - возвращаем категорию
    if 'автомобиль' in lemma_purpose:
        return 'автомобиль'
    elif ('жилье' in lemma_purpose) or ('недвижимость' in lemma_purpose):
        return 'недвижимость'
    elif 'свадьба' in lemma_purpose:
        return 'свадьба'
    elif 'образование' in lemma_purpose:
        return 'образование'
    elif 'ремонт' and 'жилье' in lemma_purpose:
        return 'ремонт'
    else:
        return 'другое'
# применяем к ДФ функцию, чтобы получить новый столбец с категориями
# axis = 1 - передаем строки
data['purpose_category'] = data.apply(purpose_category, axis = 1)

data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,have_children,total_income_category,purpose_category
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,есть дети,верхний средний класс,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,есть дети,предсредний класс,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,нет детей,предсредний класс,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,есть дети,верхний средний класс,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,нет детей,средний класс,свадьба


Столбец с категорией по целям добавлен, идем дальше.

Теперь рассмотрим столбец **debt**. Определим уникальные значения и их количество.

In [65]:
display(data['debt'].value_counts())

0    19598
1     1732
Name: debt, dtype: int64

Выведем сгруппированное количество строк по категории ежемесячного дохода, когда значение **debt** равно 1:

In [66]:
data[data['debt'] == 1].groupby('total_income_category')['debt'].count().sort_values(ascending = False)

total_income_category
предсредний класс        656
средний класс            528
нижний средний класс     350
верхний средний класс    180
состоятельные             12
выше бедности              4
богатые                    2
Name: debt, dtype: int64

Теперь когда **debt** равен 0:

In [67]:
data[data['debt'] == 0].groupby('total_income_category')['debt'].count().sort_values(ascending = False)

total_income_category
предсредний класс        7096
средний класс            5810
нижний средний класс     4019
верхний средний класс    2394
состоятельные             185
выше бедности              71
богатые                    23
Name: debt, dtype: int64

Из нашей сформулированной категории ежемесячного дохода, количество богатых (доход выше 1 млн. р.) составляет всего два человека при значении 1. В то время, как среди богатых при значении 0 составляет 23 человека. Более того, количество клиентов, у которых значение столбца 0 больше приблизительно в 10 раз по сравнению со значению 1.
Отсюда сформулируется следующее обозначение столбца **debt**:
* `0` - не имел задолженность
* `1` - имел задолженность

Теперь создадим новый столбец с категорией наличия задолженности через метод apply, создав заранее функцию, которая будет возвращать категориальную переменную в зависимости от значения **debt**.

In [68]:
def debt_category(debt):
    if debt == 0:
        return 'не имел задолженность'
    else:
        return 'имел задолженность'
data['debt_category'] = data['debt'].apply(debt_category)
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,have_children,total_income_category,purpose_category,debt_category
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,есть дети,верхний средний класс,недвижимость,не имел задолженность
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,есть дети,предсредний класс,автомобиль,не имел задолженность
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,нет детей,предсредний класс,недвижимость,не имел задолженность
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,есть дети,верхний средний класс,образование,не имел задолженность
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,нет детей,средний класс,свадьба,не имел задолженность


**Вывод**: в данном разделе определили на какие категории будем разделять наши полученные значения и добавили их в основную таблицу.

## Ответы на вопросы <a id='qs'></a>
[в начало](#begin)

- Есть ли зависимость между наличием детей и возвратом кредита в срок? <a id='q1'></a>
[в начало](#begin)

Для того, чтобы определить зависимость между наличием детей и возвратом кредита в срок необходимо составить сводную таблицу из наших сформированных категориальных переменных.

В качестве строк, по которым будем группировать наши значения возьмем столбец с категорией **debt_category**. Столбец, по значениям которого будет производиться группировка будет столбец **have_children**, в качестве значений будем брать столбец **debt**.

In [69]:
data_children_debt = data.pivot_table(index = 'debt_category', columns = 'have_children', values = 'debt', aggfunc = 'count')
display(data_children_debt)

have_children,есть дети,нет детей
debt_category,,
имел задолженность,669,1063
не имел задолженность,6571,13027


Посчитаем долю должников, у которых есть дети. 

In [70]:
debt_1_children_yes = round(data_children_debt['есть дети'][0] / (data_children_debt['есть дети'][0] + data_children_debt['есть дети'][1])*100,2)
print('Доля должников с детьми:', debt_1_children_yes)
debt_0_children_yes = round(data_children_debt['есть дети'][1] / (data_children_debt['есть дети'][0] + data_children_debt['есть дети'][1])*100,2)
print('Доля людей без долгов с детьми:', debt_0_children_yes)

Доля должников с детьми: 9.24
Доля людей без долгов с детьми: 90.76


Теперь посчитаем долю должников, у которых нет детей.

In [71]:
debt_1_children_no = round(data_children_debt['нет детей'][0] / (data_children_debt['нет детей'][0] + data_children_debt['нет детей'][1])*100,2)
print('Доля должников без детей:', debt_1_children_no)
debt_0_children_no = round(data_children_debt['нет детей'][1] / (data_children_debt['нет детей'][0] + data_children_debt['нет детей'][1])*100,2)
print('Доля людей без долгов и без детей:', debt_0_children_no)

Доля должников без детей: 7.54
Доля людей без долгов и без детей: 92.46


**Вывод**
На основе сводной таблицы и расчетов можно сказать:

* Клиенты с детьми реже берут кредиты
* Клиенты с детьми чаще не имеют задолженности среди должников

- Есть ли зависимость между семейным положением и возвратом кредита в срок? <a id='q2'></a>
[в начало](#begin)

Для того, чтобы определить зависимость между семейным статусом и возвратом кредита в срок необходимо составить сводную таблицу из наших сформированных категориальных переменных.

In [72]:
data_family_debt = data.pivot_table(index = ['family_status_id', 'family_status'], columns = 'debt_category', values = 'debt', aggfunc = 'count')
display(data_family_debt)

,debt_category,имел задолженность,не имел задолженность
family_status_id,family_status,,
0,женат / замужем,927,11334
1,гражданский брак,385,3748
2,вдовец / вдова,63,888
3,в разводе,84,1105
4,Не женат / не замужем,273,2523


Посчитаем долю клиентов-должников в каждой семейной категории.

In [73]:
for elem in range(0, 5):
    count = 0 
    count =  round(data_family_debt['имел задолженность'][elem] / 
                   (data_family_debt['имел задолженность'][elem] + 
                    data_family_debt['не имел задолженность'][elem])*100 ,2)
    print('Доля должников из общего количества клиентов в категории:', count)

Доля должников из общего количества клиентов в категории: family_status
женат / замужем    7.56
dtype: float64
Доля должников из общего количества клиентов в категории: family_status
гражданский брак    9.32
dtype: float64
Доля должников из общего количества клиентов в категории: family_status
вдовец / вдова    6.62
dtype: float64
Доля должников из общего количества клиентов в категории: family_status
в разводе    7.06
dtype: float64
Доля должников из общего количества клиентов в категории: family_status
Не женат / не замужем    9.76
dtype: float64


**Вывод**
На основе сводной таблицы и расчетов можно сказать следующее: 
* Наибольшую долю кредитов берут клиенты с семейным положением "женат/замужем"
* Наименьшую долю кредитов берут клиенты с семейным положением "вдовец/вдова"
* Люди с семейным статусом "Не жентат / не замужем" больше всех склонны не платить по кредиту вовремя
* Наибольшую долю из всех должников представляют клиенты с семейным статусом "женат/замужем"

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <a id='q3'></a>
[в начало](#begin)

Для того, чтобы определить зависимость между уровнем дохода и возвратом кредита в срок необходимо составить сводную таблицу из наших сформированных категориальных переменных.

In [74]:
data_income_debt = data.pivot_table(index = 'total_income_category', columns = 'debt_category', values = 'debt', aggfunc = 'count')
display(data_income_debt)

debt_category,имел задолженность,не имел задолженность
total_income_category,,
богатые,2,23
верхний средний класс,180,2394
выше бедности,4,71
нижний средний класс,350,4019
предсредний класс,656,7096
состоятельные,12,185
средний класс,528,5810


Посчитаем долю клиентов-должников из всего количества клиентов в категории.

In [75]:
for elem in range(0, 7):
    count = 0 
    count =  round(data_income_debt['имел задолженность'][elem] / 
                   (data_income_debt['имел задолженность'][elem] + 
                    data_income_debt['не имел задолженность'][elem])*100 ,2)
    print('Доля должников из общего количества клиентов в категории:', count)

Доля должников из общего количества клиентов в категории: 8.0
Доля должников из общего количества клиентов в категории: 6.99
Доля должников из общего количества клиентов в категории: 5.33
Доля должников из общего количества клиентов в категории: 8.01
Доля должников из общего количества клиентов в категории: 8.46
Доля должников из общего количества клиентов в категории: 6.09
Доля должников из общего количества клиентов в категории: 8.33


**Вывод**

На основе сводной таблицы и расчетов можно сказать следующее: 
* Больше всего кредиты берут клиенты предсреднего класса
* Меньше всего кредиты берут богатые клиенты
* Среди всех должников больше всего клиентов из предсреднего класса
* Среди всех должников меньше всего клиентов из богатого класса
* Люди предсреднего класса больше всех из категории не платят за кредит вовремя
* Люди класса выше бедности стараются оплачивать кредит вовремя лучше всех

- Как разные цели кредита влияют на его возврат в срок? <a id='q4'></a>
[в начало](#begin)

Для того, чтобы определить зависимость между целями кредита и его возвратом в срок необходимо составить сводную таблицу из наших сформированных категориальных переменных.

In [76]:
data_purpose_debt = data.pivot_table(index = 'purpose_category', columns = 'debt_category', values = 'debt', aggfunc = 'count')
display(data_purpose_debt)

debt_category,имел задолженность,не имел задолженность
purpose_category,,
автомобиль,400,3879
недвижимость,780,9970
образование,369,3619
свадьба,183,2130


Определим долю должников-клиентов по каждой цели.

In [77]:
purpose_list = ['автомобиль', 'недвижимость', 'образование', 'свадьба']
for elem in range(0, 4):
    count = 0 
    count =  round(data_purpose_debt['имел задолженность'][elem] / 
                   (data_purpose_debt['имел задолженность'][elem] + 
                    data_purpose_debt['не имел задолженность'][elem])*100 ,2)
    print(f'Доля должников из общего количества клиентов в категории {purpose_list[elem]}:', count)

Доля должников из общего количества клиентов в категории автомобиль: 9.35
Доля должников из общего количества клиентов в категории недвижимость: 7.26
Доля должников из общего количества клиентов в категории образование: 9.25
Доля должников из общего количества клиентов в категории свадьба: 7.91


**Вывод**

На основе расчетов и сводной таблицы можно сказать следующее:
* Больше всего кредитов берут на недвижиомость
* Среди всех должников, доля долгов за недвижимость больше всех
* Меньше всего кредиты берут на свадьбу
* Меньше всего должников в своей категории относятся клиенты, которые берут кредит на недвижимость.

## Общий вывод <a id='total'></a>
[в начало](#begin)

В процессе знакомства с данными - **статистика плтажеспособности клиентов**. Были выявлены следующие проблемы в данных:
* **явные** пропуски в количественных переменных (**days_employed** , **total_income**);
* отсутствие дополнительной информации, раскрывающий смысл значений, содержащихся в них (**education_id**, **family_status_id**, **debt**);
* наличие в столбцах аномальных значений, искажающих представление о них для тех, кто работает с ними (**days_employed**, **children**, **dob_years**, **total_income**, **age**);
* наличие дубликатов в виде значений столбца **education** разного регистра (верхний/нижний/вместе).

В процессе предобработки данных было проведено следующее:
* Некоторые столбцы были убраны, так как для проведения исследования они не нужны (**days_employed**)
* явные пропуски были заполнены соответствующими значениями (**total_income** - медианным значением дохода клиентов)
* неявный пропуск в столбце **gender** был удален
* аномальные значения в некоторых столбцах были удалены (**children**, **age**)
* дубликаты в исходных данных были удалены
* значения столбца **education** были приведены к общему виду;
* в процессе категоризации провели разбор значений, смысл которых был неизвестен (**family_status_id**).

После предобработки данных в исходную таблицу с клиентами были добавлены столбцы, значения которых представляют собой категории клиентов в зависимости от предмета исследования.
* для столбца **children** была добавлена категория **have_children**, показывающая наличие детей у клиента;
* для столбца **total_income** была добавлена категория **total_income_category**, демонстрирующая доход клиентов по категориям;
* для столбца **debt** была добавлена категория **debt_category**, характеризующая наличие долга у клиента
* для семейного статус клиента в исходной таблице были данные в виде признаков **family_status**, **family_status_id**, значения которых привели к единому соответствию
* для столбца **purpose** был добавлен столбец **purpose_category**, которая содержит в себе цели кредита клиентов по категориям.

Кроме того, в процессе категоризации была проведена операция лемматизации, позволяющая привести разные формулировки целей кредита клиента к общей категории.

В процессе исследования были сделаны сводные таблицы, которые объединили в себе необходимые для анализа признаки. На основе полученных данных из сводных таблиц были выявлены следующие закономерности: 
* наличие детей влияет на оплату кредита в срок
* семейного статус влияет на оплату кредита в срок
* финансовое положение (статус) влияет на оплату кредита в срок
* в зависимости от целей кредита люди по-разному относятся к своим долгам.

**Рекомендации заказчику**:
1. привести заполнение пропусков к общему виду, например к значению "неизвестно";
2. Добавить дополинительную информацию к исходной таблице данных к категориальным переменным, чтобы было понятно, с чем мы работаем;
4. доработать систему сбора статистики в части корректности заполнения столбцов **total_income** и **days_employed**, добавить дополнительную информацию о том, в каких единицах измерения представлены эти признаки;
3. привести заполнение значений столбцов разного регистра к общему виду.

**К исследовательской части проекта:**
* среди клиентов в категории **наличие детей** клиенты **с детьми** чаще всего не имеют задолженности по кредиту, однако такие клиенты реже берут кредиты;
* среди клиентов в категории **семейный статус** клиенты **заммужем/женат** больше всех берут кредиты, однако среди должников это самая большая группа людей;
* среди клиентов в категории **доход**  клиенты **предсреднего класса** с ежемесячным доходом **от 100 до 150 т.р** больше всех берут кредиты, однако их доля среди всех должников больше всего.
* среди клиентов в категории **цель кредита** клиенты обращаются в банк за кредитом, связанным с **недвижимостью**, среди всех должников клиентов с такой целью кредита больше всего.